In [ ]:
# Drive mount
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/gopinath-balu/computer_vision

In [ ]:
# 모듈 임포트
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# 원본 동영상 Display
video = io.open('/content/gdrive/MyDrive/CV/Face Detection/data/video/son.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data = '''<video width='50%' controls>
                    <source src = 'data:video/mp4;base64,{0}' type = 'video/mp4'/>
                    </video>'''.format(encoded.decode('ascii')))

In [ ]:
# DNN 학습 모델(caffemodel) 사용 정의
model_name = '/content/computer_vision/CAFFE_DNN/res10_300x300_ssd_iter_140000.caffemodel'

# 모델 Architecture 정의
prototxt_name = '/content/computer_vision/CAFFE_DNN/deploy.prototxt.txt'

# Detection 최소 확률(신뢰도) 50% 정의
min_confidence = 0.5

# 원본 동영상 정의
file_name = '/content/gdrive/MyDrive/CV/Face Detection/data/video/son.mp4'

# detection 결과물(output 동영상) 이름 정의
output_name = 'son_output_video.mp4'

In [ ]:
# Face Detection & Display 함수 정의
def detectAndDisplay(frame):

    # caffemodel의 weight 값과 모델 네트워크 구성을 불러와서 모델 정의
    model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)

    # 이미지를 300x300 으로 size를 조정하고 blob 를 만든다.
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, 
                                 (300, 300), (104.0, 177.0, 123.0))

    # blob을 모델에 넣는다
    model.setInput(blob)

    # detection을 수행
    detections = model.forward()

    # detections 한 수만큼 루프가 돈다.
    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]  # confidence 는 detection한 확률을 나타냄

        # min_confidence 보다 큰 경우에만 detection으로 인정함
        if confidence > min_confidence:
            (height, width) = frame.shape[:2]
            
            # detection 된 영역을 boxing
            # 상대적 좌표 * np.array([width, height, width, height]) 절대적인 boxing 좌표 구하기
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = box.astype("int")
            
            # print(confidence, startX, startY, endX, endY)

            # 얼굴에 bounding box(사각형)를 그리고 확률값 표시
            text = '{:.2f}%'.format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
    # video를 disk 에 output 하기 위해 writer 초기화
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)
        
    # disk에 frame을 write
    if writer is not None:
        writer.write(frame)

In [ ]:
# 원본 동영상에서 video stram 읽기
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('- -(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('- -(!) No Captured Frame - - Break!')
        break
    detectAndDisplay(frame)